# Data cleaning

# Preprocessing

In [50]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("../data/raw/datos-sismicos.xlsx")

## Correcting the type of values

In [3]:
for i in df.columns:
    print("- ", i, ": ", df[i].dtype)

-  fecha UTC :  str
-  hora UTC :  str
-  latitud (º) :  float64
-  longitud (º) :  float64
-  profundidad (km) :  int64
-  magnitud (M) :  float64


In [4]:
# "fecha UTC" should be date type, and "hora UTC" should be hour

fecha = df["fecha UTC"].str.strip()

hora = df["hora UTC"].str.strip().str.split(".", n=1).str[0]

df["fecha"] = pd.to_datetime(
                    fecha + " " + hora,
                    format= "%Y-%m-%d %H:%M:%S",
                    utc= True
)

df = df[["fecha", "latitud (º)", "longitud (º)", "profundidad (km)", "magnitud (M)"]]

## Adjusting the column names

In [5]:
df = df.rename(columns={
        'fecha': 'date', 
        'latitud (º)':'latitude',
        'longitud (º)': 'longitude',
        'profundidad (km)': 'depth',
        'magnitud (M)': 'magnitude'
        }
    )

## Turning the hour to Peruvian Timezone

In [6]:
df["date"] = df["date"].dt.tz_convert("America/Lima")

## First Export

In [7]:
df.to_csv("../data/processed/earthquake_clean.csv", index=False)

# Final Processing for Analysis

In [8]:
df = pd.read_csv("../data/processed/earthquake_clean.csv")

df["date"] = pd.to_datetime(df["date"], utc=True)
df["date"] = df["date"].dt.tz_convert("America/Lima")

## Date separation

In [9]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour

## Location

* Departments

In [10]:
import geopandas as gpd

deps = gpd.read_file("../data/raw/DEPARTAMENTOS_LIMITES")

In [11]:
gdf = gpd.GeoDataFrame(
    df,
    geometry = gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

In [12]:
deps = deps.to_crs(gdf.crs)

gdf = gpd.sjoin(gdf, deps, how="left", predicate="intersects")

In [13]:
df["department"] = gdf["DEPARTAMEN"]

df = df[["date", "depth", "magnitude", "year", "month", "day", "hour", "department", "longitude", "latitude"]]

In [14]:
# Changing the nan for surroundings of the country, because those earthquakes impact on Peru too

df = df.fillna({"department": "surroundings"})

* Regions

In [30]:
gdf = gpd.GeoDataFrame(
    df,
    geometry = gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

reg = gpd.read_file("../data/raw/region-geografica")

In [31]:
print(reg.crs == gdf.crs)

False


In [32]:
reg = reg.to_crs(gdf.crs)

In [33]:
gdf = gpd.sjoin(gdf, reg, how="left", predicate="intersects")

* Ocean

In [34]:
oceans = gpd.read_file("../data/raw/World-Ocean-ShapeFile")

print(oceans.crs == gdf.crs)

True


In [35]:
oceans.head()

,featurecla,scalerank,min_zoom,geometry
0,Ocean,0,0,"MULTIPOLYGON (((59.91603 -67.40049, 59.67856 -..."


In [36]:
gdf = gdf.drop(columns=["index_right"], errors="ignore") #avoids conflict with the other join

gdf = gpd.sjoin(gdf, oceans, how="left", predicate="intersects")

In [ ]:
integrating_ocean = gdf[gdf["nombre"].isna()]

,date,depth,magnitude,year,month,day,hour,department,longitude,latitude,...,situaciÃ³n,id,name_strin,key,parent_id,name_es,index_right,featurecla,scalerank,min_zoom
3,1960-01-22 22:37:32-05:00,300,5.8,1960,1,22,22,surroundings,-68.500,-12.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1960-03-09 18:54:25-05:00,80,6.2,1960,3,9,18,surroundings,-73.817,-16.389,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Ocean,0.0,0.0
9,1960-05-03 20:28:52-05:00,100,5.0,1960,5,3,20,surroundings,-71.500,-18.000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Ocean,0.0,0.0
10,1960-05-12 07:09:24-05:00,60,5.8,1960,5,12,7,surroundings,-72.500,-9.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1960-07-04 03:02:07-05:00,600,6.2,1960,7,4,3,surroundings,-71.000,-8.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:

gdf.loc[gdf["nombre"].isna(), "nombre"] = integrating_ocean["featurecla"]

In [41]:
df["region"] = gdf["nombre"]

In [47]:
df["region"] = df["region"].fillna("Bordering Country")

## Asigning categories

### Grouping by magnitude

In [51]:
bins = [-np.inf, 3, 4, 5, 6, 7, 8, np.inf]
labels = ["Micro","Minor","Light","Moderate","Strong","Major","Great"]

df["type_magnitude"] = pd.cut(
    df["magnitude"],
    bins = bins,
    labels = labels
)

print(df["type_magnitude"].value_counts())

type_magnitude
Light       18817
Moderate     3856
Minor        2216
Strong        208
Major          26
Micro           1
Great           1
Name: count, dtype: int64


### Grouping by time

In [52]:
# Grouping by quarter of the year

bins = [0, 3, 6, 9, 12]
labels = ["Q1", "Q2", "Q3", "Q4"]

df["quarter"] = pd.cut(
    df["month"],
    bins = bins,
    labels = labels,
)

print(df["quarter"].value_counts())

quarter
Q4    6712
Q2    6281
Q1    6097
Q3    6035
Name: count, dtype: int64


In [53]:
# Grouping by time of the day
bins = [-1, 6, 12, 18, 24]
labels = ["Night", "Morning", "Afternoon", "Evening"]

df["time_of_day"] = pd.cut(
    df["hour"],
    bins = bins,
    labels = labels,
)

print(df["time_of_day"].value_counts())

time_of_day
Night        8164
Morning      5734
Evening      5634
Afternoon    5593
Name: count, dtype: int64


In [54]:
# Grouping by day of the week
bins = [-1, 0, 1, 2, 3, 4, 5, 6]
labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df["day_of_week"] = pd.cut(
    df["date"].dt.dayofweek,
    bins = bins,
    labels = labels,
)

print(df["day_of_week"].value_counts())

day_of_week
Sunday       3768
Saturday     3660
Wednesday    3641
Thursday     3589
Monday       3555
Friday       3501
Tuesday      3411
Name: count, dtype: int64


In [55]:
print(df["year"].max())
print(df["year"].min())

2026
1960


In [58]:
# Grouping by decade
bins = [1960, 1970, 1980, 1990, 2000, 2010, 2020, np.inf]
labels = ["1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

df["decade"] = pd.cut(
    df["year"],
    bins = bins,
    labels = labels,
    right=False
)

print(df["decade"].value_counts())


decade
2010s    7963
1990s    4596
2000s    4528
2020s    4057
1980s    1853
1970s    1126
1960s    1002
Name: count, dtype: int64


### Grouping by depth

In [59]:
print(df["depth"].max())
print(df["depth"].min())

743
5


In [60]:
bins = [0, 70, 300, np.inf]
labels = ["Shallow", "Intermediate", "Deep"]

df["type_depth"] = pd.cut(
    df["depth"], 
    bins = bins, 
    labels = labels
)

print(df["type_depth"].value_counts())

type_depth
Shallow         16175
Intermediate     8802
Deep              148
Name: count, dtype: int64


## Fixing readability

In [62]:
df["department"] = df["department"].str.title()

In [67]:
df["region"] = df["region"].replace({'Costa': 'Coastal Region', 'Sierra': 'Andean Highlands', 'Selva': 'Amazon Rainforest', 'Ocean': 'Pacific Ocean'})

In [69]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 25125 entries, 0 to 25124
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype                       
---  ------          --------------  -----                       
 0   date            25125 non-null  datetime64[us, America/Lima]
 1   depth           25125 non-null  int64                       
 2   magnitude       25125 non-null  float64                     
 3   year            25125 non-null  int32                       
 4   month           25125 non-null  int32                       
 5   day             25125 non-null  int32                       
 6   hour            25125 non-null  int32                       
 7   department      25125 non-null  str                         
 8   longitude       25125 non-null  float64                     
 9   latitude        25125 non-null  float64                     
 10  region          25125 non-null  str                         
 11  type_magnitude  25125 non-null  categor

In [70]:
df.to_csv("../data/processed/earthquake_analysis_ready.csv", index=False)